In [5]:
import sentencepiece as spm
import librosa
import numpy as np
from pydub import AudioSegment
import speech_recognition as sr
import os

In [4]:
# Initialize the recognizer
recognizer = sr.Recognizer()

# Load an audio folder
audio_folder = "ai_assistant\\training_data"
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".mp3")
               
for i in audio_folder:

               audio_path = os.path.join(audio_folder, audio
               
               
            
                # Transcribe audio to text
                with sr.AudioFile(i) as source:
                    audio = recognizer.record(source)  # Record the entire audio file
                    transcription = recognizer.recognize_google(audio)  # Use Google's ASR
                    
                # file_count += 1
                # Save the transcription to a text file
                transcription_file = "transcription_{i}.txt"
                with open(transcription_file, "w") as file:
                    file.write(transcription)
                
                # Print a confirmation message
                print(f"Transcription {i} saved to {transcription_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'a'

In [ ]:
# Step 1: Load Audiobook
audio = AudioSegment.from_file("audiobook.mp3")

# Step 2: Text-Audio Alignment (using Aeneas )
# Assume the alignments are obtained and saved in a list of tuples: alignments
# Each tuple contains the start and end times (in milliseconds) for each audio chunk
# alignments = [(start1, end1), (start2, end2), ...]

# Also, assume transcription.txt contains the corresponding text for each audio chunk
# Each line in transcription.txt corresponds to an audio chunk

# Step 3: Train SentencePiece model
spm.SentencePieceTrainer.train('--input=transcription.txt --model_prefix=m --vocab_size=2000')
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# Step 4: Preprocess Data
audio_chunks = [audio[start:end] for start, end in alignments]

# Read the text data from transcription.txt
with open('transcription.txt', 'r') as file:
    texts = [line.strip() for line in file]

# Tokenize text
tokenized_texts = [sp.encode(text, out_type=str) for text in texts]

# Step 5: Feature Engineering
def extract_features(audio_chunk):
    samples = np.array(audio_chunk.get_array_of_samples())
    spectrogram = librosa.feature.melspectrogram(y=samples, sr=audio_chunk.frame_rate)
    return spectrogram

spectrograms = [extract_features(audio_chunk) for audio_chunk in audio_chunks]

# Step 6: Create Dataset
dataset = list(zip(spectrograms, texts, tokenized_texts))

# Step 7: Model Selection and Training
# Choose a TTS model architecture (e.g., Tacotron 2)
# Set up the training environment and train the model on the prepared dataset
# The exact steps will depend on the TTS framework and model you choose

# Step 8: Evaluation
# Evaluate the performance of your TTS model
# The evaluation steps will also depend on the TTS framework and model you choose



In [1]:
from TTS.tts.configs.bark_config import BarkConfig
from TTS.tts.models.bark import Barkg

ModuleNotFoundError: No module named 'TTS'

In [ ]:
processor = BarkProcessor.from_pretrained("suno/bark")


train_dataset = datasets.load_dataset("")


def preprocess_data(batch):
    audio = batch["audio"]
    text = batch["text"]
    inputs = processor(text, audio, return_tensors="pt", padding=True)
    return inputs

train_dataset = train_dataset.map(preprocess_data)

model = BarkForPreTraining.from_pretrained("suno/bark")

# training params
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

# initialise the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda data: {"input_values": data["input_values"], "labels": data["labels"]},
    train_dataset=train_dataset,
)


trainer.train()